In [185]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pydot

In [186]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_url(text): 
    url_pattern  = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub(r'', text)
 # converting return value from list to string



def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.lower()

In [187]:
train_data= pd.read_csv("train.csv")
train_data.dropna(axis = 0, how ='any',inplace=True) 
train_data['Num_words_text'] = train_data['message'].apply(lambda x:len(str(x).split())) 
mask = train_data['Num_words_text'] >2
train_data = train_data[mask]
max_train_sentence_length  = train_data['Num_words_text'].max()


train_data['message'] = train_data['message'].apply(remove_emoji)
train_data['message'] = train_data['message'].apply(remove_url)
train_data['message'] = train_data['message'].apply(clean_text)

In [188]:
num_words = 20000

tokenizer = Tokenizer(num_words=num_words,oov_token="unk")
tokenizer.fit_on_texts(train_data['message'].tolist())


In [189]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data['message'].tolist(),\
                                                      train_data['result'].tolist(),\
                                                      test_size=0.1,\
                                                      stratify = train_data['result'].tolist(),\
                                                      random_state=0)

x_train = np.array( tokenizer.texts_to_sequences(X_train) )
x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )

x_train = pad_sequences(x_train, padding='post', maxlen=40)
x_valid = pad_sequences(x_valid, padding='post', maxlen=40)

le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels))
valid_labels = le.transform(y_valid)
valid_labels = np.asarray( tf.keras.utils.to_categorical(valid_labels))

list(le.classes_)


train_ds = tf.data.Dataset.from_tensor_slices((x_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


In [190]:
train_labels = le.fit_transform(y_train)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels))

In [193]:
max_features =20000
embedding_dim =64
sequence_length = 40

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features +1, embedding_dim, input_length=sequence_length,\
                                    embeddings_regularizer = regularizers.l2(0.0005)))                                    

model.add(tf.keras.layers.Conv1D(128,3, activation='relu',\
                                 kernel_regularizer = regularizers.l2(0.0005),\
                                 bias_regularizer = regularizers.l2(0.0005)))                               


model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(3, activation='sigmoid',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))

model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer='Nadam', metrics=["CategoricalAccuracy"])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 40, 64)            1280064   
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 38, 128)           24704     
_________________________________________________________________
global_max_pooling1d_12 (Glo (None, 128)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 387       
Total params: 1,305,155
Trainable params: 1,305,155
Non-trainable params: 0
_________________________________________________________________


In [194]:
epochs = 100
history = model.fit(train_ds.shuffle(2000).batch(128),
                    epochs= epochs ,
                    validation_data=valid_ds.batch(128),
                    verbose=1)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4870: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '


189/189 [==============================] - 10s 50ms/step - loss: 1.0452 - categorical_accuracy: 0.5557 - val_loss: 0.8094 - val_categorical_accuracy: 0.6883
Epoch 2/100
189/189 [==============================] - 9s 45ms/step - loss: 0.7963 - categorical_accuracy: 0.7038 - val_loss: 0.7754 - val_categorical_accuracy: 0.7201
Epoch 3/100
189/189 [==============================] - 9s 46ms/step - loss: 0.7585 - categorical_accuracy: 0.7305 - val_loss: 0.7795 - val_categorical_accuracy: 0.7250
Epoch 4/100
189/189 [==============================] - 8s 45ms/step - loss: 0.7305 - categorical_accuracy: 0.7468 - val_loss: 0.7956 - val_categorical_accuracy: 0.7253
Epoch 5/100
189/189 [==============================] - 9s 45ms/step - loss: 0.7190 - categorical_accuracy: 0.7612 - val_loss: 0.8118 - val_categorical_accuracy: 0.7283
Epoch 6/100
189/189 [==============================] - 9s 48ms/step - loss: 0.7015 - categorical_accuracy: 0.7813 - val_loss: 0.8251 - val_categorical_accuracy: 0.7268
Epo

In [195]:
history.history

{'categorical_accuracy': [0.5556986331939697,
  0.7038128972053528,
  0.7305200099945068,
  0.7467602491378784,
  0.7611729502677917,
  0.7812759876251221,
  0.7983884215354919,
  0.8111812472343445,
  0.8299551606178284,
  0.8378052711486816,
  0.8498920202255249,
  0.8592789769172668,
  0.8664230108261108,
  0.870908796787262,
  0.8785927891731262,
  0.8832447528839111,
  0.8912194967269897,
  0.8935039043426514,
  0.8966190218925476,
  0.9019355177879333,
  0.9065043926239014,
  0.9083319306373596,
  0.9085811376571655,
  0.9105333089828491,
  0.9106994271278381,
  0.9152683019638062,
  0.9170958399772644,
  0.9202525615692139,
  0.9191311001777649,
  0.9214985966682434,
  0.9218723773956299,
  0.9239076375961304,
  0.9263997077941895,
  0.926192045211792,
  0.9263166785240173,
  0.9287257194519043,
  0.9306778311729431,
  0.9310932159423828,
  0.933169960975647,
  0.9327130913734436,
  0.9332530498504639,
  0.9352882504463196,
  0.9353713393211365,
  0.9346652030944824,
  0.9370742

In [223]:
model.save('tf_cnnmodel') 
json_string = tokenizer.to_json()

phoneUrl1 = "https://www.bajajfinservmarkets.in/emi-store/lg-w10-32-gb-smoky-gray-3-gb-ram-smartphone.html"
phoneUrl2 = "https://www.bajajfinservmarkets.in/emi-store/oppo-a15s-64-gb-fancy-white-4-gb-ram-smartphone.html"
null = "NULL"

INFO:tensorflow:Assets written to: tf_cnnmodel/assets


In [224]:
import json
with open('tokenizer.json', 'w') as outfile:
    json.dump(json_string, outfile)

In [225]:
new_model = tf.keras.models.load_model('tf_cnnmodel')
new_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 40, 64)            1280064   
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 38, 128)           24704     
_________________________________________________________________
global_max_pooling1d_12 (Glo (None, 128)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 387       
Total params: 1,305,155
Trainable params: 1,305,155
Non-trainable params: 0
_________________________________________________________________


In [226]:
with open('tokenizer.json') as json_file:
    json_string = json.load(json_file)
tokenizer1 = tf.keras.preprocessing.text.tokenizer_from_json(json_string)

In [ ]:
labels = [phoneUrl1, phoneUrl2, null]

message = input('Enter the sentence ')
x_test  = tokenizer1.texts_to_sequences([message])
x_test = pad_sequences(x_test, padding='post', maxlen=40)

predictions = new_model.predict(x_test)
predict_results = predictions.argmax(axis=1)
print(labels[int(predict_results)])